In [4]:
import json, os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set(style="whitegrid")

In [7]:
stats = {}
with open("alien_ecology_stats/stats.json", "r") as f:
    stats = json.loads(f.read())

show = {'environment temperature':
        ['env_temp'],
        'food stats':
        ['food'],
        'mean_age':
        ['prev_learning_age_mean',
         'prev_evolving_age_mean',
         'gs_evolving_age_mean',
         'gs_learning_age_mean',],
        'mean_fitness':
        ['prev_learning_fitness_mean',
         'prev_evolving_fitness_mean',
         'gs_evolving_fitness_mean', 
         'gs_learning_fitness_mean'],
        'distance moved':
        ['prev_learning_distance_mean',
         'prev_evolving_distance_mean',
         'gs_evolving_distance_mean',
         'gs_learning_distance_mean'],
        'best agent spread':
        ['learned_in_top_gs', 
         'evolved_in_top_gs',
         'learned_in_top_prev',
         'evolved_in_top_prev'],
        'learned action distribution':
        ['learning_rotate_right',
         'learning_rotate_left',
         'learning_propel',
         'learning_pick_food',
         'learning_plant_food',
         'learning_eat_food',
         'learning_mate',
         'learning_freq_up',
         'learning_freq_down',
         'learning_emit_pheromone',
         'learning_move_random'],
         'evolved action distribution':
        ['evolving_rotate_right',
         'evolving_rotate_left',
         'evolving_propel',
         'evolving_pick_food',
         'evolving_plant_food',
         'evolving_eat_food',
         'evolving_mate',
         'evolving_freq_up',
         'evolving_freq_down',
         'evolving_emit_pheromone',
         'evolving_move_random']}

for title, group in show.items():
    min_p = min(len(stats[l]) for l in group)
    plot_data = {}
    for label in group:
        plot_data[label] = stats[label][-min_p:]
    data = pd.DataFrame(plot_data)
    fig = plt.figure(figsize=(15,8))
    data = data.rolling(10).mean()
    ax = sns.lineplot(data=data, hue="event", style="event", dashes=False, markers=False, linewidth=1.5)
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax.set_title(title)
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'alien_ecology_stats/stats.json'